# Shor's Algorithm

In [ ]:
from cirq_qubitization.jupyter_tools import show_bloq

In [ ]:
from cirq_qubitization.bloq_algos.shors import *
from cirq_qubitization.bloq_algos.shors.shors import *

In [ ]:
import numpy as np
from sympy import S

## Factor 13 * 17

 - `N` is the composite number to factor
 - `n` is its bitsize

In [ ]:
N = 13*17
n = int(np.ceil(np.log2(N)))
N, n

Pick a random guess to serve as the base of our exponentiation

In [ ]:
g = 8
g

### Period Finding

Find the period of the exponentiation -- namely how long it takes to cycle back to 1 under modular arithmetic.

In [ ]:
for e in range(20):
    f = (g ** e) % N
    
    star = ' *' if f == 1 else ''
    print(f'{e:5d} {f:5d}{star}')

In [ ]:
# The period is indeed consistent
16-8, 8-0

In [ ]:
period = 8

### Use the period to find factors

We can use some numerical tricks to use our period to find two factors.

In [ ]:
assert period %2 == 0
assert g**(period//2) != -1

half_period = g**(period//2)
p1 = half_period + 1
m1 = half_period - 1

assert (p1*m1) % N == 0

In [ ]:
print(f'gcd{p1%N, N}, gcd{m1%N, N}')
import math
math.gcd(p1%N, N), math.gcd(m1%N, N)

## Bloqs

We'll do modular exponentiation like above with `g` and `N`. `exponent` is a $2n$ sized input and we will allocate a new $n$ sized register `x` which will contain the output.

In [ ]:
bloq = ModExp.make_for_shor(big_n=N, g=g)
show_bloq(bloq)

In [ ]:
from sympy import Symbol
bloq = ModExp.make_for_shor(big_n=Symbol('N'), g=Symbol('g'))
show_bloq(bloq)

### Smaller example

In [ ]:
N = 3*5
n = int(np.ceil(np.log2(N)))
g = 8

for e in range(20):
    f = (g ** e) % N
    star = ' *' if f == 1 else ''
    print(f'{e:5d} {f:5d}{star}')

In [ ]:
bloq = ModExp.make_for_shor(big_n=N, g=g)
show_bloq(bloq)

In [ ]:
from cirq_qubitization.bloq_algos.shors.shors import big_endian_bits_to_int, int_to_bits

for e in range(20):
    out_regs = bloq.apply_classical(exponent=e)
    e = out_regs['exponent']
    f = out_regs['x']
    star = ' *' if f == 1 else ''
    print(f'{e:5d} {f:5d}{star}')

In [ ]:
from cirq_qubitization.quantum_graph.graphviz import ClassicalSimGraphDrawer
ClassicalSimGraphDrawer(bloq, {'exponent': 5}).get_svg()

In [ ]:
cbloq = bloq.decompose_bloq()
show_bloq(cbloq)

In [ ]:
[pow(g, int(2**j), N) for j in range(8)]

In [ ]:
ClassicalSimGraphDrawer(cbloq, {'exponent': 5}).get_svg()

In [ ]:
cbloq.apply_classical(exponent=5)

In [ ]:
from cirq_qubitization.bloq_algos.shors.shors import big_endian_bits_to_int, int_to_bits

for ei in range(20):
    out_regs = bloq.apply_classical(exponent=ei)
    out_regs2 = cbloq.apply_classical(exponent=ei)
    
    e = out_regs['exponent']
    f = out_regs['x']
    star = ' *' if f == 1 else ''
    
    e2 = out_regs2['exponent']
    f2 = out_regs2['x']
    
    print(f'{e:5d} {f:5d}{star:2s}  {e2:5d} {f2:5d}')

In [ ]:
cmm = CtrlModMul(k=S('k'), x_bitsize=5, mod_N=N)
print(cmm)
show_bloq(cmm.decompose_bloq())